In [1]:
import numpy as np
import pandas as pd
from sergio import sergio

### Dataset

In [2]:
dataset = {
    "DS9": {
            "number_genes": 100,
            "number_bins" : 3,
            "number_sc": 300,
            "bifurcation_matrix": "data/DS9/bMat_cID9.tab",
            "GRN": "data/DS9/Interaction_cID_9.txt",
            "master_regulator_prod_rate": "data/DS9/Regs_cID_9.txt",
            "save_location": "data/DS9/SERGIO_sim_DS9.npy"
            },
    "DS10": {
            "number_genes": 100,
            "number_bins" : 4,
            "number_sc": 300,
            "bifurcation_matrix": "data/DS10/bMat_cID10.tab",
            "GRN": "data/DS10/Interaction_cID_10.txt",
            "master_regulator_prod_rate": "data/DS10/Regs_cID_10.txt",
            "save_location": "data/DS10/SERGIO_sim_DS10.npy"
            },
    "DS12": {
            "number_genes": 100,
            "number_bins" : 7,
            "number_sc": 300,
            "bifurcation_matrix": "data/DS12/bMat_cID12.tab",
            "GRN": "data/DS12/Interaction_cID_12.txt",
            "master_regulator_prod_rate": "data/DS12/Regs_cID_12.txt",
            "save_location": "data/DS12/SERGIO_sim_DS12.npy"
            }
}

### Steady-State Simulation

In [3]:
sim_id = "DS10"

In [4]:
sim = sergio(number_genes = dataset[sim_id]["number_genes"], 
             number_bins = dataset[sim_id]["number_bins"], 
             number_sc = dataset[sim_id]["number_sc"],  
             noise_params = 0.1, 
             decays = 0.8, 
             sampling_state = 15, 
             noise_type = 'dpd')

In [5]:
sim.build_graph(dataset[sim_id]["GRN"], 
                input_file_regs = dataset[sim_id]["master_regulator_prod_rate"], 
                shared_coop_state = 2)
sim.simulate()
expr = sim.getExpressions()
expr_clean = np.concatenate(expr, axis = 1)

Start simulating new level
There are 3 genes to simulate in this layer
Done with current level
Start simulating new level
There are 7 genes to simulate in this layer
Done with current level
Start simulating new level
There are 90 genes to simulate in this layer
Done with current level


### Steady-State Simulation (add noise)

In [6]:
"""
Add outlier genes
"""
expr_O = sim.outlier_effect(expr, outlier_prob = 0.01, mean = 0.8, scale = 1)

"""
Add Library Size Effect
"""
libFactor, expr_O_L = sim.lib_size_effect(expr_O, mean = 4.6, scale = 0.4)

"""
Add Dropouts
"""
binary_ind = sim.dropout_indicator(expr_O_L, shape = 6.5, percentile = 82)
expr_O_L_D = np.multiply(binary_ind, expr_O_L)

"""
Convert to UMI count
"""
count_matrix = sim.convert_to_UMIcounts(expr_O_L_D)

"""
Make a 2d gene expression matrix
"""
count_matrix = np.concatenate(count_matrix, axis = 1)

### Differentiation Simulation (without noise)

In [7]:
df = pd.read_csv(dataset[sim_id]["bifurcation_matrix"], 
                 sep='\t', header=None, index_col=None)
bMat = df.values

sim = sergio(number_genes = dataset[sim_id]["number_genes"], 
             number_bins = dataset[sim_id]["number_bins"], 
             number_sc = dataset[sim_id]["number_sc"], 
             noise_params = 0.2, 
             decays = 0.8, 
             sampling_state = 1, 
             noise_params_splice = 0.07, 
             noise_type = 'dpd', 
             dynamics = True, 
             bifurcation_matrix = bMat)

sim.build_graph(dataset[sim_id]["GRN"], 
                input_file_regs = dataset[sim_id]["master_regulator_prod_rate"], 
                shared_coop_state = 2)

In [8]:
sim.simulate_dynamics()
exprU, exprS = sim.getExpressions_dynamics()
exprU_clean = np.concatenate(exprU, axis = 1)
exprS_clean = np.concatenate(exprS, axis = 1)

Start simulating new cell type
binID: 0
number of initial cells: 25
Done with current cell type
Start simulating new cell type
binID: 1
number of initial cells: 27
Done with current cell type
Start simulating new cell type
binID: 3
number of initial cells: 300
Done with current cell type
Start simulating new cell type
binID: 2
number of initial cells: 27
Done with current cell type


### Differentiation Simulation (add noise)

In [9]:
"""
Add outlier genes
"""
exprU_O, exprS_O = sim.outlier_effect_dynamics(exprU, exprS, outlier_prob = 0.01, mean = 0.8, scale = 1)

"""
Add Library Size Effect
"""
libFactor, exprU_O_L, exprS_O_L = sim.lib_size_effect_dynamics(exprU_O, exprS_O, mean = 4.6, scale = 0.4)

"""
Add Dropouts
"""
binary_indU, binary_indS = sim.dropout_indicator_dynamics(exprU_O_L, exprS_O_L, shape = 6.5, percentile = 82)
exprU_O_L_D = np.multiply(binary_indU, exprU_O_L)
exprS_O_L_D = np.multiply(binary_indS, exprS_O_L)

"""
Convert to UMI count
"""
count_matrix_U, count_matrix_S = sim.convert_to_UMIcounts_dynamics(exprU_O_L_D, exprS_O_L_D)

"""
Make 2d spliced and unspliced expression matrices
"""
count_matrix_U = np.concatenate(count_matrix_U, axis = 1)
count_matrix_S = np.concatenate(count_matrix_S, axis = 1)

In [10]:
sim_data = {"steady_state_expr": expr,
            "steady_state_expr_noisy": count_matrix,
            "unspliced_expr": exprU,
            "spliced_expr": exprS,
            "unspliced_expr_noisy": count_matrix_U,
            "spliced_expr_noisy": count_matrix_S
           } 

np.save(dataset[sim_id]["save_location"], sim_data)